In [1]:
import urllib.request
import os.path
import pandas as pd
import numpy as np

from nameparser import HumanName
import humanize


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [2]:
if not os.path.isfile("2021_Contributions.csv"):
    url = "https://www.nyccfb.info/DataLibrary/2021_Contributions.csv"
    urllib.request.urlretrieve(url, "2021_Contributions.csv")

In [3]:
donations = pd.read_csv("2021_Contributions.csv", header=0, dtype={ 'NAME': str, 'STRNO': str, 'STRNAME': str, 'APARTMENT': str , 'INTZIP': str, 'EXEMPTCD': str})

In [4]:
donations

,ELECTION,OFFICECD,RECIPID,CANCLASS,RECIPNAME,COMMITTEE,FILING,SCHEDULE,PAGENO,SEQUENCENO,REFNO,DATE,REFUNDDATE,NAME,C_CODE,STRNO,STRNAME,APARTMENT,BOROUGHCD,CITY,STATE,ZIP,OCCUPATION,EMPNAME,EMPSTRNO,EMPSTRNAME,EMPCITY,EMPSTATE,AMNT,MATCHAMNT,PREVAMNT,PAY_METHOD,INTERMNO,INTERMNAME,INTSTRNO,INTSTRNM,INTAPTNO,INTCITY,INTST,INTZIP,INTEMPNAME,INTEMPSTNO,INTEMPSTNM,INTEMPCITY,INTEMPST,INTOCCUPA,PURPOSECD,EXEMPTCD,ADJTYPECD,RR_IND,SEG_IND,INT_C_CODE
0,2021,1,2563,P,"Tirschwell, Sara A",H,7,ABC,NaN,NaN,R0000209,3/6/2021,NaN,"""Dombrowski,"", Ray",IND,NaN,NaN,NaN,NaN,Haverford,PA,19041,Turnaround Manager,Alvarez & Marsal,NaN,600 Madison Avenue,New York,NY,2000.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
1,2021,55,2345,P,"Camarena, Rodrigo G",H,6,ABC,NaN,NaN,R0001664,12/17/2020,NaN,"(Dougherty) LoBianco, Kelly",IND,NaN,NaN,NaN,Z,Los Angeles,CA,90068,Chief Program Officer,The HOPE Program,1,Smith Street,Brooklyn,NY,38.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
2,2021,55,2414,P,"Low, Jenny L",H,6,ABC,NaN,NaN,R0000080,9/5/2020,NaN,"(Eng) Lawton, Linnit",IND,NaN,NaN,NaN,K,Brooklyn,NY,11201,Human Resources,Hamilton Madison House,253,South Street,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
3,2021,55,283,P,"Gennaro, James F",M,7,ABC,NaN,NaN,R0002295,3/3/2021,NaN,"(Orlow) Husarsky, Miriam",IND,NaN,NaN,NaN,Q,Richmond Hill,NY,11418,Speech Therapist,NYC Dept of Education,63-55,102nd street,Rego Park,NY,175.0,175,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
4,2021,55,2454,P,"Boghosian Murphy, Leslie",H,6,ABC,NaN,NaN,R0000171,10/19/2020,NaN,"(Zimmerman)Shaich, Carol",IND,NaN,NaN,NaN,M,Ny,NY,10027,Retired,NaN,NaN,NaN,NaN,NaN,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285576,2021,11,2546,P,"Yang, Andrew",H,7,ABC,NaN,NaN,R0029434,1/13/2021,NaN,"Zymaris, Mai",IND,NaN,NaN,NaN,Z,BRAINTREE,MA,2184,Attorney,Morrison & Foerster,428,John Mahar Hwy Unit 207,BRAINTREE,MA,50.0,0,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
285577,2021,11,2393,P,"Wiley, Maya D",H,7,ABC,NaN,NaN,R0034934,2/13/2021,NaN,"Zysermann, Noemie",IND,NaN,NaN,NaN,M,New York,NY,10023,Consultant,Self Employed,266,W 73rd st,New York,NY,25.0,25,0.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN
285578,2021,33,1559,P,"Johnson, Corey D",L,3,ABC,NaN,NaN,R0016683,7/11/2019,NaN,"Zyskowska, Celina",IND,NaN,NaN,NaN,Q,Kew Gardens,NY,11415,Accounting,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,250.0,250,0.0,2,16.0,"Oneill, Brendan",NaN,NaN,NaN,Long Island City,NY,11104,Commodore Construction,602,South 3rd Avenue,Mt Vernon,NY,Labor Manager,NaN,NaN,NaN,N,N,IND
285579,2021,11,1545,P,"Adams, Eric L",L,1,ABC,NaN,NaN,R0003574,6/12/2018,NaN,"Zyskowski, Joseph",IND,NaN,NaN,NaN,Q,Middle Village,NY,11379,Owner,K Construction,65-58,Admiral Avenue,Middle Village,NY,5000.0,250,0.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,N,NaN


In [5]:
major_candidates = [
    'Adams, Eric L',
    'Donovan, Shaun',
    'Garcia, Kathryn A',
    'McGuire, Raymond J',
    'Morales, Dianne',
    'Stringer, Scott M',
    'Wiley, Maya D',
    'Yang, Andrew'
]

mayor_candidates_names = donations[donations['OFFICECD'].isin([1, 11])]['RECIPNAME'].unique()
candidate_last_names = dict(zip(mayor_candidates_names, map(lambda s: HumanName(s).last, mayor_candidates_names)))
candidate_last_names.values()

dict_values(['Tirschwell', 'Wiley', 'Yang', 'Morales', 'Stringer', 'Cullen', 'Garcia', 'McGuire', 'Donovan', 'Menchaca', 'Adams', 'Diaz', 'Pepitone', 'Taylor', 'Sutton', 'Rose', 'Mateo', 'Guimaraes', 'Chang', 'Sliwa', 'Wright', 'Emilien', 'Filipchenko', 'Laurel-Smith', 'Foldenauer', 'Kavovit', 'Francis', 'Prince', 'Nunez', 'Kaplan', 'Oremland', 'Miles', 'Reaves', 'Coenen', 'Bunea', 'Pinto', 'Downs', 'Flores', 'Fitzgerald', 'Krietchman', 'Nampiaparampil', "O'Hagan", 'Seidman', 'Seely'])

In [6]:
# mayor_donations = donations[ (donations['OFFICECD'].isin([1, 11]) & donations[donations["RECIPNAME"].isin(major_candidates) & donations["SCHEDULE"].eq("ABC") ]
mayor_donations = donations[ donations['OFFICECD'].isin([1, 11]) & donations["RECIPNAME"].isin(major_candidates) & donations["SCHEDULE"].eq("ABC") ].copy()

In [7]:
len(mayor_donations)
1500/len(mayor_donations) * 100

1.8236416908805757

In [8]:
mayor_donations.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82253 entries, 24 to 285579
Data columns (total 52 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ELECTION    82253 non-null  int64  
 1   OFFICECD    82253 non-null  int64  
 2   RECIPID     82253 non-null  object 
 3   CANCLASS    82253 non-null  object 
 4   RECIPNAME   82253 non-null  object 
 5   COMMITTEE   82253 non-null  object 
 6   FILING      82253 non-null  int64  
 7   SCHEDULE    82253 non-null  object 
 8   PAGENO      0 non-null      float64
 9   SEQUENCENO  0 non-null      float64
 10  REFNO       82253 non-null  object 
 11  DATE        82253 non-null  object 
 12  REFUNDDATE  0 non-null      object 
 13  NAME        82253 non-null  object 
 14  C_CODE      82253 non-null  object 
 15  STRNO       0 non-null      object 
 16  STRNAME     0 non-null      object 
 17  APARTMENT   0 non-null      object 
 18  BOROUGHCD   82172 non-null  object 
 19  CITY        82179 non-n

In [9]:
mayor_donations.NAME.value_counts()

Koo, Julia             53
Jordan, Tyron          45
Warren, Roxanne        44
Sherrill, Kenneth      36
Yourke, George         32
                       ..
Arrendell, Jane         1
Williams,, Reginald     1
Wing Li, Gang           1
McPhail, Marlinda       1
Mc Queen, Larry         1
Name: NAME, Length: 55702, dtype: int64

In [10]:
mayor_donations['C_CODE'].unique()

array(['PCOMP', 'CORP', 'IND', 'CAN', 'FAM', 'OTHR', 'PCOMZ', 'EMPO',
       'LLC'], dtype=object)

In [11]:
def parse_name(s):
    names = []
    
    human_name = HumanName(s)
    
    names.append(f"{human_name.last}, {human_name.first}")
    
    if human_name.suffix:
        names.append(f"{human_name.last} {human_name.suffix}, {human_name.first}")
    
    if human_name.middle:
        names.append(f"{human_name.last}, {human_name.first} {human_name.middle}")
    
        if len(human_name.middle) > 1:
            names.append(f"{human_name.last}, {human_name.first} {human_name.middle[0]}")
    
    return names


def name_variations_from_file(file):
    variations = []
    
    with open(file, 'r') as f:
        for line in f:
            for variation in parse_name(line):
                variations.append(variation.upper())

    return variations
                
real_estate_names_from_littlesis = name_variations_from_file('./real_estate_names.txt')
worst_landlords = name_variations_from_file('./worst_landlords.txt')

In [12]:
mayor_donations['realestate_littlesis'] = mayor_donations['NAME'].str.upper().isin(real_estate_names_from_littlesis)
mayor_donations['worst_landlord'] = mayor_donations['NAME'].str.upper().isin(worst_landlords)
mayor_donations['works_in_realestate'] = mayor_donations['OCCUPATION'].str.contains(r'(?i).*REAL ESTATE.*')

In [13]:
flagged_donations = mayor_donations[ (mayor_donations['realestate_littlesis'] | mayor_donations['worst_landlord'] |  mayor_donations['works_in_realestate']  | ( mayor_donations['C_CODE'] == "LLC"))]
flagged_donations_major_candidates = flagged_donations[flagged_donations["RECIPNAME"].isin(major_candidates)]

columns = ['ELECTION', 'RECIPID', 'RECIPNAME', 'COMMITTEE', 'NAME', 'AMNT', 'DATE', 'REFUNDDATE', 'CITY', 'STATE', 'ZIP', 'OCCUPATION', 'EMPNAME', 'C_CODE', 'realestate_littlesis', 'worst_landlord', 'works_in_realestate']

flagged_donations.sort_values(['RECIPNAME', 'DATE'])[columns]

,ELECTION,RECIPID,RECIPNAME,COMMITTEE,NAME,AMNT,DATE,REFUNDDATE,CITY,STATE,ZIP,OCCUPATION,EMPNAME,C_CODE,realestate_littlesis,worst_landlord,works_in_realestate
49376,2021,1545,"Adams, Eric L",L,"Cohen, Adir",5100.0,1/10/2019,NaN,Brooklyn,NY,11230,Real Estate,Renaissance Realty Group,IND,False,False,True
96693,2021,1545,"Adams, Eric L",L,"Gottlieb, Israel",1000.0,1/10/2019,NaN,Lakewood Township,NJ,8701,Real estate investor,Self-Employed,IND,False,False,True
164427,2021,1545,"Adams, Eric L",L,"Matheus, Donald",500.0,1/10/2019,NaN,Brooklyn,NY,11238,Real estate,Albatross group,IND,False,False,True
249109,2021,1545,"Adams, Eric L",L,"Strulovitch, Baila",2000.0,1/10/2019,NaN,Brooklyn,NY,11205,Real Estate,Self Employee,IND,False,False,True
148543,2021,1545,"Adams, Eric L",L,"Levy, Danny",100.0,1/10/2020,NaN,Brooklyn,NY,11230,Real estate,Self-Employed,IND,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6634,2021,2546,"Yang, Andrew",H,"Alpert, Joseph",1000.0,3/9/2021,NaN,Fort Lee,NJ,7024,Real Estate,Self Employed,IND,False,False,True
19718,2021,2546,"Yang, Andrew",H,"Bellini, Arthur",2000.0,3/9/2021,NaN,New York,NY,10012,Real Estate,Black Bear Asset Management,IND,False,False,True
43526,2021,2546,"Yang, Andrew",H,"Chen, Cindy",10.0,3/9/2021,NaN,Brooklyn,NY,11201,Real estate agent,Compass,IND,False,False,True
85071,2021,2546,"Yang, Andrew",H,"Friedman, Joseph",250.0,3/9/2021,NaN,Brooklyn,NY,11210,Real estate,Self Employed,IND,False,False,True


In [14]:
flagged_donations.sort_values(['RECIPNAME', 'DATE']).to_csv('flagged_donations.csv')

In [15]:
def to_money(i):
 return "$" + humanize.intcomma(round(i))

def to_person_name(s):
    return str(HumanName(s))

    
# candidate_donations = donations[donations["RECIPNAME"].isin(major_candidates) & donations["SCHEDULE"].eq("ABC")]


In [16]:
total_amount_recieved = mayor_donations[["RECIPNAME", "AMNT"]].groupby("RECIPNAME").sum().sort_values(by="AMNT", ascending=False)
total_amount_recieved["AMNT"]  = total_amount_recieved["AMNT"].apply(to_money)
total_amount_recieved = total_amount_recieved.rename_axis('Candidates').rename(columns={'AMNT': 'Amount'})
total_amount_recieved

,Amount
Candidates,
"McGuire, Raymond J","$7,645,510"
"Adams, Eric L","$3,807,982"
"Stringer, Scott M","$2,555,446"
"Donovan, Shaun","$2,320,507"
"Yang, Andrew","$2,174,180"
"Wiley, Maya D","$1,057,497"
"Garcia, Kathryn A","$586,578"
"Morales, Dianne","$562,751"


In [17]:
def largest_donors_per_candidate(donations, limit=15):
    agg = donations[["RECIPNAME", "AMNT", "NAME", "EMPNAME"]].groupby(["RECIPNAME", "NAME"]).agg({'AMNT': sum, 'EMPNAME': lambda s: s.unique() }) # .sort_values(by='AMNT', ascending=False) # .groupby(level=0, group_keys=False).head(20) # .reset_index() # ort_values(by='AMNT')  # .groupby(level=0, group_keys=False).apply(lambda x: x.sort_values(('AMNT', 'sum'), ascending=False))
        
    
    return agg['AMNT'].groupby('RECIPNAME', group_keys=False).apply(lambda x: x.sort_values(ascending=False).head(limit).apply(to_money)).to_frame()
    

In [18]:
largest_donors_per_candidate(mayor_donations)

AMNT
RECIPNAME          NAME                                
Adams, Eric L      Bensusan, Danny              $12,100
                   Waisman, Shai                $10,200
                   Guzel, Murat                 $10,100
                   Boutross, Diana              $10,000
                   Singh, Navdeep               $10,000
                   Serova, Daria                $10,000
                   Bartlett, Gregory             $7,600
                   Akcetin, Mehmet               $7,000
                   David, Emanuel                $6,600
                   Cerchione, Gregory            $6,000
                   Razi, Sami Uddin              $6,000
                   Basusta, Yusuf                $5,500
                   Daneshvar, Michelle           $5,350
                   engelbrecht, Kate             $5,100
                   Belt, Antonia                 $5,100
Donovan, Shaun     Donovan, Shaun               $19,100
                   Blumenstein, Ruth            $15,300
                   Briganti, Stephen            $10,200
                   Zabar, Eli                   $10,200
                   Nolen, Eliot                 $10,200
                   Rockefeller, David           $10,200
                   Fredericks, Devon            $10,200
                   Bingham, Clara               $10,200
                   Goodman, Sara                $10,150
                   Niejelow, Alexander           $8,100
                   Hickenlooper, Robin           $8,000
                   Whang, John                   $6,100
                   Meyer, Audrey                 $5,600
                   Werner, Zachary               $5,600
                   Donovan, Lucas                $5,450
Garcia, Kathryn A  Smith, Daniel                 $2,250
                   sando, vicki                  $2,135
                   Hughes, Chris                 $2,000
                   Berardi, Eugene               $2,000
                   Wilson, Frederick             $2,000
                   Hansen, Monika                $2,000
                   Glazer, Elizabeth             $2,000
                   Wachsberger, Chaim            $2,000
                   Wilson, Joanne                $2,000
                   Kerr, Zachary                 $2,000
                   Kelly, Philip                 $2,000
                   MacArthur, Andrew             $2,000
                   Russo, Michael                $2,000
                   Uniformed Sanitation Chiefs   $2,000
                   Lettieri, Greg                $2,000
McGuire, Raymond J Williams-Isom, Anne          $15,300
                   Rubin, Robert                $10,200
                   Tisch, James                 $10,200
                   Lutnick, Allison             $10,200
                   Gerson, Mark                 $10,200
                   Rose, Joe                    $10,200
                   Abramson, Carrie             $10,200
                   Fuller, Vicki                $10,200
                   Adjmi, Marcus                $10,200
                   Ross, Kara                   $10,200
                   Chenault, Kenneth            $10,200
                   Atkins, Peter                $10,200
                   Wolfe, Lisa                  $10,200
                   Schnell, Paul                $10,200
                   Kempner, Michael             $10,100
Morales, Dianne    Weinberg, Jason               $6,000
                   momeyer, alan                 $2,100
                   Little, Keith                 $2,100
                   Shavers, Frances              $2,030
                   Murray, Brian                 $2,000
                   Kovner, Katherine             $2,000
                   Murray, Edward                $2,000
                   Schnapper, Simeon             $2,000
                   Wainright, Hannah             $2,000
                   Johnson, William              $2,000
                   

In [28]:
largest_donors_per_candidate(mayor_donations, limit=3)

AMNT
RECIPNAME          NAME                        
Adams, Eric L      Bensusan, Danny      $12,100
                   Waisman, Shai        $10,200
                   Guzel, Murat         $10,100
Donovan, Shaun     Donovan, Shaun       $19,100
                   Blumenstein, Ruth    $15,300
                   Briganti, Stephen    $10,200
Garcia, Kathryn A  Smith, Daniel         $2,250
                   sando, vicki          $2,135
                   Hughes, Chris         $2,000
McGuire, Raymond J Williams-Isom, Anne  $15,300
                   Rubin, Robert        $10,200
                   Tisch, James         $10,200
Morales, Dianne    Weinberg, Jason       $6,000
                   momeyer, alan         $2,100
                   Little, Keith         $2,100
Stringer, Scott M  Guenther, Diane A    $10,200
                   Klein, John S        $10,100
                   Redhead, Fabian       $9,900
Wiley, Maya D      Essex, Andrew         $4,100
                   Carter, Zachary       $4,000
                   Wang, Elizabeth       $3,500
Yang, Andrew       Koo, Julia            $8,877
                   Hu, Jordan            $4,000
                   DiCristo, Aurelia     $4,000

In [19]:
# Largest Donors
largest_donors_per_candidate(mayor_donations)

AMNT
RECIPNAME          NAME                                
Adams, Eric L      Bensusan, Danny              $12,100
                   Waisman, Shai                $10,200
                   Guzel, Murat                 $10,100
                   Boutross, Diana              $10,000
                   Singh, Navdeep               $10,000
                   Serova, Daria                $10,000
                   Bartlett, Gregory             $7,600
                   Akcetin, Mehmet               $7,000
                   David, Emanuel                $6,600
                   Cerchione, Gregory            $6,000
                   Razi, Sami Uddin              $6,000
                   Basusta, Yusuf                $5,500
                   Daneshvar, Michelle           $5,350
                   engelbrecht, Kate             $5,100
                   Belt, Antonia                 $5,100
Donovan, Shaun     Donovan, Shaun               $19,100
                   Blumenstein, Ruth            $15,300
                   Briganti, Stephen            $10,200
                   Zabar, Eli                   $10,200
                   Nolen, Eliot                 $10,200
                   Rockefeller, David           $10,200
                   Fredericks, Devon            $10,200
                   Bingham, Clara               $10,200
                   Goodman, Sara                $10,150
                   Niejelow, Alexander           $8,100
                   Hickenlooper, Robin           $8,000
                   Whang, John                   $6,100
                   Meyer, Audrey                 $5,600
                   Werner, Zachary               $5,600
                   Donovan, Lucas                $5,450
Garcia, Kathryn A  Smith, Daniel                 $2,250
                   sando, vicki                  $2,135
                   Hughes, Chris                 $2,000
                   Berardi, Eugene               $2,000
                   Wilson, Frederick             $2,000
                   Hansen, Monika                $2,000
                   Glazer, Elizabeth             $2,000
                   Wachsberger, Chaim            $2,000
                   Wilson, Joanne                $2,000
                   Kerr, Zachary                 $2,000
                   Kelly, Philip                 $2,000
                   MacArthur, Andrew             $2,000
                   Russo, Michael                $2,000
                   Uniformed Sanitation Chiefs   $2,000
                   Lettieri, Greg                $2,000
McGuire, Raymond J Williams-Isom, Anne          $15,300
                   Rubin, Robert                $10,200
                   Tisch, James                 $10,200
                   Lutnick, Allison             $10,200
                   Gerson, Mark                 $10,200
                   Rose, Joe                    $10,200
                   Abramson, Carrie             $10,200
                   Fuller, Vicki                $10,200
                   Adjmi, Marcus                $10,200
                   Ross, Kara                   $10,200
                   Chenault, Kenneth            $10,200
                   Atkins, Peter                $10,200
                   Wolfe, Lisa                  $10,200
                   Schnell, Paul                $10,200
                   Kempner, Michael             $10,100
Morales, Dianne    Weinberg, Jason               $6,000
                   momeyer, alan                 $2,100
                   Little, Keith                 $2,100
                   Shavers, Frances              $2,030
                   Murray, Brian                 $2,000
                   Kovner, Katherine             $2,000
                   Murray, Edward                $2,000
                   Schnapper, Simeon             $2,000
                   Wainright, Hannah             $2,000
                   Johnson, William              $2,000
                   

In [20]:
# Largest Real Estate Donors
largest_donors_per_candidate(flagged_donations_major_candidates, limit=30)

AMNT
RECIPNAME          NAME                              
Adams, Eric L      Boutross, Diana            $10,000
                   tamir, Elliot               $5,100
                   Berman, Nathan              $5,100
                   Verrone, Robert             $5,100
                   Tawil, Eliot                $5,100
                   Demirjian, Alexis           $5,100
                   Cohen, Adir                 $5,100
                   Borrok, Charles R.          $5,100
                   Shurka, Manny               $5,000
                   Schwartz, Yoel              $5,000
                   Corman, Jeffrey             $5,000
                   Landau, Solomon             $5,000
                   Neamonitakis, Michael       $4,000
                   Ward, Jesse                 $3,400
                   Vitucci Sr, Dominick        $3,400
                   Fisher, Avi                 $3,400
                   Silviano, John              $3,000
                   Anagnost, Dick              $3,000
                   Levkov, Viacheslav          $3,000
                   Gindi, jeffrey              $2,675
                   Iannucci, Robert            $2,500
                   Ewers, Sonia                $2,500
                   Marchi, Dean                $2,500
                   Chassin, Jonathan           $2,500
                   Sciortino, Anthony          $2,500
                   Basile, Rocco               $2,250
                   Ghershenzon, Mordechai      $2,000
                   Chalme, Raymond             $2,000
                   Sanjana, Aroza              $2,000
                   Cohen, Meir                 $2,000
Donovan, Shaun     Fisher, Winston             $5,100
                   DeBode, Gary                $5,100
                   olshan, andrea              $5,100
                   Bahar, Aaron                $5,100
                   KATZ, JEFFREY               $5,100
                   Moore, John                 $5,100
                   Sheller, Chris              $5,100
                   Bernstein, Joshua           $5,100
                   Silvers, Nicholas           $5,100
                   Brodsky, Daniel             $5,100
                   Silverstein, Larry          $5,100
                   Stacom, Darcy               $5,100
                   Zeiler, John                $5,100
                   Winn, Arthur                $5,100
                   Clark, Ric                  $5,100
                   Martin, Rob                 $3,000
                   Rubler, Neil                $2,500
                   Benenson, Lawrence          $2,500
                   kuriloff, josh              $2,500
                   Wiener, Alan                $2,250
                   Savitt, Robert              $2,000
                   Wiener, Matthew             $2,000
                   Kerr, Zachary               $2,000
                   Willard, Sara               $1,750
                   Murray, Brian               $1,500
                   Mandel, Barry               $1,500
                   Happel, Michael             $1,100
                   Fisher, Beth                $1,000
                   Shapiro, Ben                $1,000
                   Farman-Farmaian, Patricia   $1,000
Garcia, Kathryn A  Kalikow, Peter              $2,000
                   Kerr, Zachary               $2,000
                   Durst, Helena               $2,000
                   MacArthur, Andrew           $2,000
                   Ezra, Brian                 $1,400
                   Wark, Jesse                 $1,050
                   Peterson, Douglas             $750
                   Simkowitz, Chaim              $750
                   Ghossein, Yara                $750
                   Fruchthandler, Efraim         $750
                   Nigrelli, Randy               $700
                   Amiri, Behzad                 $500
                   Cottingham, Martin            $500

In [29]:
largest_donors_per_candidate(flagged_donations_major_candidates, limit=3)

AMNT
RECIPNAME          NAME                       
Adams, Eric L      Boutross, Diana     $10,000
                   tamir, Elliot        $5,100
                   Berman, Nathan       $5,100
Donovan, Shaun     Fisher, Winston      $5,100
                   DeBode, Gary         $5,100
                   olshan, andrea       $5,100
Garcia, Kathryn A  Kalikow, Peter       $2,000
                   Kerr, Zachary        $2,000
                   Durst, Helena        $2,000
McGuire, Raymond J Colacino, Michael    $8,433
                   winter, benjamin     $5,100
                   Goldberg, Andrew     $5,100
Morales, Dianne    Rosenthal, Richard     $500
                   Neveloff, Jay          $400
                   Fried, Daniel          $350
Stringer, Scott M  Klein, John S       $10,100
                   Quinlan, Robert      $7,900
                   Brodsky, Daniel      $5,100
Wiley, Maya D      Khan, Fazeel         $2,000
                   Johnson, Michael     $1,000
                   Zelermyer, Karen       $750
Yang, Andrew       Korein, James        $2,000
                   Wei, Nelson          $2,000
                   DeLilli, Mark        $2,000

In [21]:
def top_donors_for(df, head=20):
    top_donors = df.groupby(["NAME"]).agg({"AMNT": 'sum', 'RECIPNAME': lambda series: list(series.unique()) }).sort_values(by="AMNT", ascending=False).head(head).reset_index()
    top_donors.rename(columns={"NAME": "Donor Name", "AMNT": "Amount", "RECIPNAME": "Supported Candidates" }, inplace=True)
    top_donors['Donor Name'] = top_donors['Donor Name'].apply(lambda x: str(HumanName(x)))
    top_donors["Amount"] = top_donors["Amount"].apply(lambda x: f"${humanize.intcomma(round(x))}" )
    top_donors["Supported Candidates"] = top_donors["Supported Candidates"].apply(lambda names: ", ".join(map(candidate_last_names.get, names)))
    return top_donors


all_top_donors = top_donors_for(mayor_donations, head=50)
top_real_estate_donors = top_donors_for(flagged_donations, head=50)


In [22]:
mayor_donations[["EMPNAME", "AMNT"]].groupby(["EMPNAME"]).agg(['sum', 'count']).sort_values(by=('AMNT', 'sum'), ascending=False).head(50)


AMNT       
                                       sum  count
EMPNAME                                          
Not Employed                    2695587.64  16605
Self Employed                    757348.87   3026
Self-Employed                    682807.35   1346
Retired                          507235.55   1322
Self                             479941.85   1271
Unemployed                       375357.31   6422
not employed                     340273.57    370
self                             176027.50    379
unemployed                       116675.50    285
Paul Weiss                       113435.00     66
Self employed                     84928.05    336
Citigroup                         75194.07     86
Self Employee                     72810.00    108
Citi                              65762.99     99
Homemaker                         61245.00     79
Skadden Arps                      59795.00     60
Not employed                      56529.50    330
Paul Weiss Rifkind Wharton & G    53496.00     29
Skadden                           41110.00     38
none                              40879.60     45
Pershing Square Capital Manage    39200.00      9
Monroe College                    38075.00     34
Google                            37437.60    252
Columbia University               37135.84    265
Sullivan & Cromwell LLP           36860.00     30
Lazard                            36600.00      9
Kirkland & Ellis LLP              35145.00     25
self employed                     34458.78     99
City of New York                  34269.06    432
Skadden Arps Slate Meagher & F    33800.00     17
NYU                               31393.69    232
Housewife                         31180.00     73
Cravath, Swaine & Moore LLP       31175.00     17
retired                           30470.00     87
Centerview Partners               29500.00      7
Kirkland & Ellis                  29400.00     16
self-employed                     27516.21     96
NYPD                              26644.00    228
Wachtell Lipton Rosen & Katz      25370.00     22
None                              25127.21     28
Compass                           24696.21     60
Mediaocean                        24161.00     46
Long Pond Capital                 23900.00      6
PJT Partners                      23600.00      9
Wachtell, Lipton, Rosen & Katz    22725.00     11
Wachtell Lipton                   22150.00     12
Standard Industries               21800.00      7
Cleary Gottlieb Steen & Hamilt    21550.00     14
Willkie Farr & Gallagher LLP      21218.00     22
Cleary Gottlieb                   21100.00     14

In [23]:
top_real_estate_donors

,Donor Name,Amount,Supported Candidates
0,Daniel Brodsky,"$10,200","Stringer, Donovan"
1,Winston Fisher,"$10,200","Stringer, Donovan"
2,John S Klein,"$10,100",Stringer
3,Diana Boutross,"$10,000",Adams
4,Michael Colacino,"$8,433",McGuire
5,Robert Quinlan,"$7,900",Stringer
6,Peter Kalikow,"$7,100","Garcia, McGuire"
7,Derek Trulson,"$7,100","Adams, McGuire"
8,Robert Verrone,"$7,100","Adams, Yang"
9,Richard Ruben,"$7,100","Adams, McGuire"


In [24]:
all_top_donors

,Donor Name,Amount,Supported Candidates
0,Shaun Donovan,"$19,100",Donovan
1,Anne Williams-Isom,"$17,300","Wiley, McGuire"
2,Ruth Blumenstein,"$15,300",Donovan
3,Steven Rattner,"$14,200","Stringer, Adams, Donovan, McGuire"
4,James Dolan,"$12,200","Adams, Donovan, McGuire"
5,Kevin Ryan,"$12,200","Donovan, McGuire, Yang"
6,James Tisch,"$12,200","Garcia, McGuire"
7,Robert Verrone,"$12,200","Adams, McGuire, Yang"
8,John Petry,"$12,200","Adams, McGuire, Yang"
9,Sara Goodman,"$12,150","Stringer, Donovan"


In [25]:
sum_from_real_estate = flagged_donations_major_candidates[['RECIPNAME', 'AMNT']].groupby('RECIPNAME').sum()
sum_from_real_estate = sum_from_real_estate.sort_values(by='AMNT', ascending=False)
sum_from_real_estate['AMNT'] = sum_from_real_estate['AMNT'].apply(to_money)

sum_from_real_estate

,AMNT
RECIPNAME,
"McGuire, Raymond J","$264,904"
"Adams, Eric L","$234,044"
"Stringer, Scott M","$149,423"
"Donovan, Shaun","$133,311"
"Yang, Andrew","$63,117"
"Garcia, Kathryn A","$30,875"
"Wiley, Maya D","$8,937"
"Morales, Dianne","$2,787"


In [26]:
to_money(flagged_donations_major_candidates['AMNT'].sum())

'$887,398'

In [27]:
flagged_donations_major_candidates['AMNT'].sum() / mayor_donations['AMNT'].sum() * 100

4.284781966891966